# EGM703 - Week 2 Practical: Hyperspectral image analysis

## Overview
In the lectures and reading this week, you've learned about hyperspectral remote sensing and a number of different methods for analyzing hyperspectral data. In this practical, we'll gain some experience working with hyperspectral data, using a few examples written in python.

## Objectives

- Open and view data using xarray
- Perform atmospheric correction using dark object subtraction
- Use spectral angle matching to compare spectral signatures and identify surfaces
- Gain some familiarity with Spectral Python (SPy), a python package for analyzing hyperspectral images.

## Data provided
In the `data` folder, you should have the following files:

- combine_spectral_library.py - a python script that can be used to add additional spectra to **spectral_library.csv**
- EO1H0380352003173110PF_MTL_L1T.TXT - the L1T ("level-1 terrain corrected") metadata file for the EO-1 image
- solar_spectra.csv - a csv file containing values of extraterrestrial (top of atmosphere) radiance as a function of wavelength
- spectral_library.csv - a csv file containing example reflectance spectra for different surface types

You'll need to download the hyperspectral data from Blackboard, or from the Google Drive link [here](https://drive.google.com/file/d/18EHJpSbkbARJ2Rt6NndBSPe9SlcYr_iO/view?usp=sharing) - be sure to save it to the `data` folder.

The datast provided is an EO-1 Hyperion image, acquired 22 June 2003, over an area near the border of Arizona and Nevada in the southwestern United States. To prepare the dataset, I have converted the original L1T DN values to spectral radiance, then combined and re-ordered the bands (by wavelength) into a single netcdf file.

## 1 Getting started
As always, we need to run the following cell to import the necessary libraries, as well as define a couple of functions that we'll use for displaying the image(s).

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
from scipy.interpolate import interp1d
import spectral as spy


def plot_rgb(ax, ds, bands, crs, variable='radiance'):
    """
    Plot an RGB composite of an image using the provided bands.
    
    Parameters
    ----------
    ax - a matplotlib Axes object
    ds - the xarray DataSet with a 'radiance' variable representing the image
    bands - a list of the three bands to display, in R, G, B order
    crs - a CRS object to pass to ax.imshow()
    variable - which variable from ds to show (default: radiance)
    
    Returns
    -------
    ax - the matplotlib Axes object
    """
    dispimg = []
    for b in bands:
        this_band = ds[variable].loc[b].values
        this_band = percentile_stretch(this_band)
        dispimg.append(this_band)
    dispimg = np.array(dispimg)
    ax.imshow(dispimg.transpose([1, 2, 0]), transform=crs, extent=[ds.x.min(), ds.x.max(), ds.y.min(), ds.y.max()])
    return ax
    

def percentile_stretch(image, pmin=0., pmax=100.):
    """
    Apply a linear percentile stretch to an image.
    
    Parameters
    ----------
    image - the input image
    pmin - the minimum percentile to use in the stretch
    pmax - the maximum percentile to use in the stretch
    
    Returns
    -------
    stretched - the stretched image band
    """
    # here, we make sure that pmin < pmax, and that they are between 0, 100
    if not 0 <= pmin < pmax <= 100:
        raise ValueError('0 <= pmin < pmax <= 100')
    # here, we make sure that the image is only 2-dimensional
    if not image.ndim == 2:
        raise ValueError('Image can only have two dimensions (row, column)')
    
    minval = np.percentile(image[image > 0], pmin)
    maxval = np.percentile(image[image > 0], pmax)
    
    stretched = (image - minval) / (maxval - minval) # stretch the image to 0, 1
    stretched[image < minval] = 0 # set anything less than minval to the new minimum, 0.
    stretched[image > maxval] = 1 # set anything greater than maxval to the new maximum, 1.
    
    return stretched

### 1.1 Loading the data

The image we'll be using in this practical is an EO-1 Hyperion image, acquired 22 June 2003. The images are terrain-corrected and radiometrically calibrated by the USGS. I have combined them into a single NetCDF file, re-scaled the values to radiance, and removed any bands that don't contain data. For more information about Hyperion images, including the different processing levels, see this [USGS link](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-earth-observing-one-eo-1-hyperion).

`xarray` doesn't automatically read all of the file from the disk. This means that once the image is opened, we also need to load the data using `Dataset.load()` - this is mostly so that we don't have to read the data from the disk every time we want to plot it.

In [ ]:
ds = xr.open_dataset('data/EO1H0380352003173110PF.nc')
ds.load() # this will load the entire image into memory - it may take a minute!

If you click on the document logo in each row of the **Coordinates** or the **Data variables**, you can display additional information about each variable, such as the units - this is one of the advantages of the netCDF format, which allows important metadata to be kept in the same file as the data itself.

### 1.2 Selecting bands using xarray

Once we have the data loaded, we can start to explore. In the output above, you should see the different coordinates - `band`, `x`, and `y`. You should also see the following data variables:

- `crs`: this is the CRS variable that tells a GIS software how to display the images
- `radiance`: these are the radiance values
- `wavelength`: this is the wavelength corresponding to each band.

Note, for example, that `radiance` has three dimensions: `band`, `y`, and `x`, while `wavelength` has only one: `band`. This also tells the index order for a given value in the `radiance` array: `ds['radiance'][0, 1000, 500]` corresponds to the first band, the 1001st row, and the 501st column.

This can be a little bit confusing here - the first band (index 0) is actually Hyperion Band 8 (because Bands 1-7 have no data). Rather than selecting by the array index, though, we can select by the `coordinate` - in this case, we would use the actual Hyperion band number. The following should give us information about Band 8:

In [ ]:
ds['radiance'].loc[8]

This returns the Band 8 radiance as a `DataArray` object - this is an array with 3381 rows and 1121 columns. In addition to the associated coordinates, you can also see that there are a number of attributes for the object: the units are W m<sup>-2</sup> sr<sup>-1</sup> µm<sup>-1</sup>, indicating that this is a spectral radiance. The `grid_mapping` attribute is something that tells our GIS software where to look to get the georeferencing information about the image - in this case, it's the `crs` variable.

If we want to see what wavelength is associated with band 8, we can again use `.loc`, but this time on the `wavelength` variable:

In [ ]:
ds['wavelength'].loc[8]

Here, you should hopefully see that the central wavelength of band 8 is 426.82 nm, which is in the violet portion of the visible spectrum.

### 1.3 Displaying a single band
Because the different `radiance` bands are **DataArrays**, we can display them using the `.plot()` method ([documentation](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.plot.html)) method. As we saw in EGM722, if we want to plot things geographically we need to first set up the axis by setting the `projection` with a `cartopy` CRS object - in this case, we'll use WGS84 UTM Zone 12N.

We'll again have a look at Band 8 to start with:

In [ ]:
utm_crs = ccrs.UTM(12) # set the projection as WGS84 UTM 12N
fig, ax = plt.subplots(1, 1, subplot_kw=dict(projection=utm_crs), figsize=(5, 8)) # create an axis with this projection

ds['radiance'].loc[8].plot(cmap='gray') # plot band 8 using .plot()

You can zoom in the image above to see the different features in the scene - you should see a large lake (Grand Wash) around halfway down the image, along with a large river just South of the lake (the Colorado River). In the Northern part of the image, you should also be able to see a number of canyons and cliffs - the geology in this area, as we'll see later on, is primarily sandstone/sedimentary rocks with some volcanics thrown in for flavor.

Note also that `.plot()` adds a title (telling us what band is being plotted), and a colorbar with a label that includes the units (!) We can of course update these labels if we want to. First, we'll look at the output of `fig.get_axes()` ([documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.figure.Figure.get_axes.html)) to see all of the **Axes** object contained within the figure - you should see two objects in the list below - one **GeoAxes** (where the band is plotted), and one **Axes** (the colorbar):

In [ ]:
fig.get_axes() # show the axes objects in the figure

In the cell below, write a line of code that retrieves the colorbar **Axes** object from the output of `fig.get_axes()`, and assign that to an object called `cax`:

In [ ]:
cax = # fill in the rest!

Now, run the following cell to update the title and colorbar label. We're changing the title to tell us what sensor the band comes from (EO-1 Hyperion), and we're changing the colorbar label in two ways:

1. making it all one line (rather than split over two lines)
2. using ([LaTeX](https://matplotlib.org/stable/users/explain/text/usetex.html)) formatting so that the units are displayed as W m<sup>-2</sup> sr<sup>-1</sup> µm<sup>-1</sup>, rather than W/m^2/sr/µm.

In [ ]:
bnum = 8 # set the band number
ax.set_title(f"EO-1 Hyperion Band {bnum}") # set a new title for the figure

cax.set_ylabel("Measured at-sensor radiance (W m$^{-2}$ sr$^{-1}$ µm$^{-1}$)") # replace the ylabel with a tex-formatted string

After running the cell above, you should see the title and labels on the figure change.

### 1.4 Displaying an RGB composite

To get a bit better overview, we can display an RGB composite image using bands 31 (660.85 nm), 20 (548.92 nm), and 11 (457.34 nm), which correspond to parts of visible red, green, and blue wavelengths, respectively.

To check this, we'll also look at one of the ways that we can select values from the There are a number of different ways to select values from `xarray.DataArray` objects (see this [documentation](https://docs.xarray.dev/en/latest/user-guide/indexing.html#quick-overview) page for more information). We've already seen one, `.loc`, which we have used to view information about a single layer or value of a `DataArray`. 

As we will see below, we can also use `.loc` to select multiple layers/values - here, let's use the thre bands that we've mentioned above (31, 20, and 11), to confirm the wavelength values that I listed above:

In [ ]:
combo = [31, 20, 11] # create a list of band coordinates

ds.wavelength.loc[combo]

Next, we can use the `plot_rgb()` function defined at the beginning of the notebook to display an RGB composite of these bands. Before running the cell, make sure to add a line that adds a title to the figure, explaining what bands this is an RGB composite of:

In [ ]:
rgb_fig, ax = plt.subplots(1, 1, figsize=(6, 10), subplot_kw=dict(projection=utm_crs))
ax = plot_rgb(ax, ds, combo, utm_crs)

ax.set_title() # update this to include a title with the band combination!

Hopefully, this looks recognizable as a (roughly) true-color image of the area. You should also hopefully recognize this as having clear atmospheric effects - why is that? If you're not sure, remember that you can always post questions in the discussion board on Blackboard.

To help see the atmospheric effects a little bit more clearly, let's use `ax.set_xlim()` ([documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_xlim.html)) and `ax.set_ylim()` ([documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_ylim.html)) to zoom in on Great Wash - the window above will change after you run this cell:

In [ ]:
ax.set_xlim(227500, 231500) # set the x-axis limits
ax.set_ylim(4009500, 4016000) # set the y-axis limits

In the figure window above, you should see Great Wash more clearly, along with the surroudning area.

### 1.4 Plotting spectral curves
We can also see how the value of a single pixel varies by band, or wavelength. `xarray` provides two main ways to access these - we can either use the image index (just like we have seen previously with a **list** or a **DataFrame**), or using the `.sel()` method ([documentation](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.sel.html)). 

In the first case, we'll see how to do this using the index. The next cell will plot the value of the radiance as a function of wavelength for the pixel in row 1950, column 500 (somewhere in the middle of the lake):

In [ ]:
row, col = 1950, 500 # set the row, column indices we want to see

fig, ax = plt.subplots(1, 1)

ax.plot(ds['wavelength'], ds['radiance'][:, row, col])

ax.set_title(f"radiance at {row}, {col}")
ax.set_xlabel('wavelength (nm)')
ax.set_ylabel('radiance (W m$^{-2}$ sr$^{-1}$ µm$^{-1}$)')

Notice how the radiance is significantly higher in the visible wavelengths (400-700 nm), dropping off substantially above about 600 or so nm - this indicates high radiance values in blue and green wavelengths, and significantly lower values in red and infrared wavelengths - much like we would expect for liquid water.

Now, let's see what real-world coordinates correspond to row 1950, column 500. Here, we use the `.y` and `.x` attributes of the **Dataset**, along with the index values, to print the results to the notebook:

In [ ]:
print(f"Row 1950 has a value of {ds.y[1950].values}")
print(f"Column 500 has a value of {ds.x[500].values}")

Row 1950, column 500 corresponds to `x`, `y` coordinates of (229800, 4011300). 

Rather than selecting the image coordinate, we can also select using the `x`,`y` coordinates for the image using `.sel()` - the following should create the same plot as above:

In [ ]:
x, y = 229800, 4011300

fig, ax = plt.subplots(1, 1)

spec = ds['radiance'].sel(x=x, y=y, method='nearest')

ax.plot(ds['wavelength'], spec)

ax.set_title(f"radiance at {x}, {y}")
ax.set_xlabel('wavelength (nm)')
ax.set_ylabel('radiance (W m$^{-2}$ sr$^{-1}$ µm$^{-1}$)')

## 2 Atmospheric Correction
Now that we have seen a few ways to view our data, we can try to correct the radiance values for atmospheric effects. At shorter wavelengths (compared to the thermal infrared) like we have here, we tend to see more atmospheric scattering, increasing significantly towards shorter wavelengths - in particular, this makes the image appear more blue. This is owing to the main ways that electromagnetic radiation scatters off of molecules and particles in the atmosphere, something we'll address in more detail during next week's lectures.

### 2.1 Band Histogram
For now, we'll look at atmospheric correction using dark object subtraction, something that was introduced (optionally) in the Week 1 practical. This technique is where we take the reflectance of an object (either an optically "dark" object, or objects in shadow) and subtract the observed radiance values for that object from the rest of the image.

Rather than searching the image for a suitable object, we're going to use a percentile approach - taking the radiance value that is only brighter than 0.5% of the image. First, we will check this by looking at the histogram for Band 8, using `plt.hist()` ([documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html)).

We'll use both `density=True` and `cumulative=True`, along with `histtype='step'`, so that what we see is the cumulative frequency for each set of values, displayed as a "step" plot:

In [ ]:
data = ds['radiance'].loc[8].values.flatten() # this gives us a vector array that we can pass to plt.hist()

fig, ax = plt.subplots(1, 1)
_ = ax.hist(data[data > 0], 20, density=True, cumulative=True, histtype='step') # make sure to take only the values > 0,
# otherwise we'll end up with a ton of 0 (nodata) values

ax.set_ylabel('cumulative frequency')
ax.set_xlabel('radiance (W m$^{-2}$ sr$^{-1}$ µm$^{-1}$)')

Note that there are a few values around 0, but most of the values seem to be between 60 and 120 W m<sup>-2</sup> sr<sup>-1</sup> µm<sup>-1</sup>. Based on this, a good estimate for the dark object radiance in this band seems to be around 60 W m<sup>-2</sup> sr<sup>-1</sup> µm<sup>-1</sup>. 

### 2.2 Finding the dark object value in each band
But, we don't want to have to do this by hand for every single band - instead, we'll use `numpy.percentile()` ([documentation](https://numpy.org/doc/2.1/reference/generated/numpy.percentile.html)) to calculate the value for us (this is, after all, one of the points of doing things programmatically).

In this cell, we'll first calculate the values for each band, then plot these values as a function of the band's wavelength:

In [ ]:
dark_obj = []
for b in ds['radiance']:
    # by selecting values where the value > 0, we ignore the nodata values
    dark_obj.append(np.percentile(b.values[b.values > 0], 0.5))

dark_obj = np.array(dark_obj)

fig, ax = plt.subplots(1, 1)

ax.plot(ds['wavelength'], dark_obj)

ax.set_ylabel('radiance (W m$^{-2}$ sr$^{-1}$ µm$^{-1}$)')
ax.set_xlabel('wavelength (nm)')

Notice how the value drops rapidly once we get through the visible wavelengths - again, this is because we expect to see the amount of atmospheric scattering drop exponentially as wavelength increases. We'll come back to these values again later when we look at individual reflectance curves.

## 3 Calculating reflectance

### 3.1 Solar radiance
Before we calculate reflectance, however, we need to know what the incident radiation is. In the `data` folder is a file called `solar_spectra.csv`, which contains values of extraterrestrial spectral irradiance downloaded from the [National Renewable Energy Laboratory](https://www.nrel.gov/grid/solar-resource/spectra-am1.5.html). These are provided as spectral irradiance in units of W m<sup>-2</sup> nm<sup>-1</sup>, which means that we need to multiply by 1000 (to convert from nm<sup>-1</sup> to µm<sup>-1</sup>), and divide by 4&pi; (to get values in W m<sup>-2</sup> sr<sup>-1</sup> µm<sup>-1</sup>) in order to compare with our satellite-derived values.

We'll also plot the dark object radiance and the radiance for our sample pixel in the lake, in order to see how they compare to the solar irradiance values.

In [ ]:
df = pd.read_csv('data/solar_spectra.csv')

etr_rad = df.etr * 1000 # convert to "per nanometer" values

fig, ax = plt.subplots(1, 1)

# note: in the line below, we divide by 4pi to get the value per steradian, to match with the units
ax.plot(df.wavelength, etr_rad / 4 / np.pi, 'b', label='solar irradiance')

ax.plot(ds['wavelength'], dark_obj, 'r', label='dark object radiance')
ax.plot(ds['wavelength'], spec, 'k', label='measured radiance')

ax.set_xlabel('wavelength (nm)')
ax.set_ylabel('spectral radiance (W m$^{-2}$ sr$^{-1}$ nm$^{-1}$)')

ax.legend()

### 3.2 Calculating reflectance using COST

Now that we have the dark object radiance for each band and the solar irradiance, we can calculate the reflectance &rho;<sub>&lambda;</sub> using the corrected radiance (_L_<sub>&lambda;</sub> - _L_<sub>dark</sub>) and the solar irradiance (_L_<sub>sun</sub>):

![the COST equation](img/cost_eqn.png)

Where _d_ is the Earth-Sun distance in [Astronomical Units](https://en.wikipedia.org/wiki/Astronomical_unit) and _&theta;_<sub>z</sub> is the solar zenith angle (to get this, we subtract the sun elevation angle found in the metadata, 65.098308, from 90). From the provided MTL.txt file (line 300), the value of the solar elevation angle is 65.098308°. Note that we also need to convert the zenith angle from degrees to radians using `np.deg2rad()` ([documentation](https://numpy.org/doc/stable/reference/generated/numpy.deg2rad.html)).

For the scene we are using, acquired 22 June 2003, the Earth-Sun distance is 152040710.84 km, or about 1.0163294 AU.

The plot below will show the reflectance values for the pixel over the lake that we have used previously:

In [ ]:
solar_zenith = np.deg2rad(90 - 65.098308) # solar zenith angle converted to radians
es_dist = 152040710.84 / 149597870.700 # earth-sun distance converted to astronomical units

refl_toa = (spec * es_dist**2 * np.pi) / (etr_rad * np.cos(solar_zenith)**2) # compute the top of atmosphere reflectance
refl = ((spec - dark_obj) * es_dist**2 * np.pi) / (etr_rad * np.cos(solar_zenith)**2) # compute reflectance after dark object subtraction

fig, ax = plt.subplots(1, 1)

ax.plot(ds['wavelength'], refl_toa, label='Top-of-Atmosphere Reflectance')
ax.plot(ds['wavelength'], refl, label='Corrected Reflectance')

ax.set_xlabel('wavelength (nm)')
ax.set_ylabel('spectral reflectance')

ax.legend()

Looking at the plot above, can you see where a few different "atmospheric windows" are located?

Next, we'll define a function, `calculate_reflectance()`, and apply it to each band's radiance values to get the reflectance in each band. Then, we'll add this variable to our dataset by first creating a **DataArray** object ([documentation](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.html#xarray.DataArray)).

In [ ]:
def calculate_reflectance(radiance, dark, etr, d, theta_z):
    return ((radiance - dark) * d**2 * np.pi) / (etr * np.cos(theta_z)**2)

reflectances = []
for ind, band in enumerate(ds['radiance']):
    this_refl = calculate_reflectance(band.values, dark_obj[ind], etr_rad[ind], es_dist, solar_zenith)
    this_refl[band.values == 0] = 0 # make sure that values that were 0 stay 0.
    this_refl[this_refl < 0] = 0.01 # set negative reflectance to a low value
    reflectances.append(this_refl)

# add a new variable to the dataset, reflectance, using the values calculated above
ds['reflectance'] = xr.DataArray(np.array(reflectances), dims=['band', 'y', 'x'])

Finally, we'll show the RGB image again, side-by-side with the atmospherically-corrected reflectance values. You should notice that the corrected image appears crisper, in addition to being significantly less blue - again, this is in part due to the increased scattering seen at shorter wavelengths.

In [ ]:
ref_fig, axs = plt.subplots(1, 2, figsize=(8, 10), subplot_kw=dict(projection=utm_crs))

axs[0] = plot_rgb(axs[0], ds, combo, utm_crs, variable='radiance')
axs[1] = plot_rgb(axs[1], ds, combo, utm_crs, variable='reflectance')

axs[0].set_title('Raw radiance')
axs[1].set_title('Atmospherically-corrected reflectance')

Comparing the two images side-by-side, you should hopefully be able to see the difference between the top-of-atmosphere reflectance and the corrected reflectance. What are the main differences that you can see?

## 4. Spectral Angle Mapping

### 4.1 Using a single end member
Next up, we'll see how we can calculate the angle between the spectral vector for our example pixel and water. We'll start by loading our spectral library samples, then calculate the spectral angle &alpha; according to the formula:

![spectral angle mapping formula](img/sam_eqn.png)

For this, we'll use [Spectral Python](https://www.spectralpython.net/) (SPy), a python module for processing hyperspectral data. In addition to spectral angle mapping, SPy also has a number of algorithms that we have discussed, including minimum noise fraction (MNF) and principal component analysis (PCA), and it also includes tools for querying the ECOSTRESS Spectral Library.

In [ ]:
# load the spectral library values that have been re-sampled to the same wavelengths as the EO-1 image.
spectral_library = pd.read_csv('data/spectral_library.csv')

water_angles = spy.spectral_angles(ds['reflectance'].values.transpose([1, 2, 0]), 
                                   spectral_library['water'].values.reshape(1, -1))

First, though, let's plot the end members in our spectral library, so we have some idea of what their spectral signatures look like:

In [ ]:
end_members = list(spectral_library.columns) # get the columns of the dataframe
end_members.remove('wavelength') # remove the wavelength column from our list of end members

fig, ax = plt.subplots(1, 1)

for mem in end_members:
    ax.plot(spectral_library.wavelength, spectral_library[mem], label=mem)

ax.set_ylabel('reflectance')
ax.set_xlabel('wavelength [nm]')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.) # add a legend

fig.tight_layout() # use this to make sure we can see the legend

We can now look at the spectral angle for each pixel - we'll focus on the lake so we can get an idea of how well it works:

In [ ]:
fig, ax = plt.subplots(1, 1)

ang_im = ax.imshow(water_angles[1800:2000, 450:540], vmin=0.5, vmax=0.8)
cax = fig.colorbar(ang_im)

cax.set_label('spectral angle')

You should see that the water has a comparatively small angle (dark colors), while the land has a typically larger angle. Overall, it seems to have worked as we might expect, although it does appear somewhat noisy.

With one spectral signature like this example, we could choose a threshold angle for binary classification - pixels with an angle less than the given threshold would be classified as water, and pixels with an angle larger than the given threshold would be classified as 'not water'

To see why the results for this example might be somewhat noisy, let's look at the reflectance spectra for a few example pixels from the lake:

In [ ]:
# we can also select a range of pixels using xarray
# in this example, we're selecting based on a range of x and y values
test_pixels = ds['reflectance'].sel(x=np.linspace(229300, 229600, 10),
                                    y=np.linspace(4011800, 4012100, 10), method='nearest')

fig, ax = plt.subplots(1, 1)
ax.plot(ds['wavelength'], test_pixels.values.reshape(194, -1), '0.5')
ax.plot(ds['wavelength'], spectral_library['water'].values, 'k')

In the example above, we can see that there's quite a bit of noise in our reflectance spectra - this is contributing to the noise we can see in the spectral angle results.

### 4.2 Multiple end members

With additional reference spectra (end members), we can find which end member each pixel is 'closest' to. With `spy.spectral_angles()` ([documentation](https://www.spectralpython.net/class_func_ref.html#spectral-angles)), we can use as many end members as we have data for. In the next example will do this for all of our reference spectra:

In [ ]:
# spectral_angles expects the data to have a shape (rows, columns, bands), while our data are (bands, rows, columns)
# we also need to transpose our spectral library data so that the rows correspond to each end member
all_angles = spy.spectral_angles(ds['reflectance'].values.transpose([1, 2, 0]), 
                                 spectral_library[end_members].values.transpose())

Using `np.argmin()` ([documentation](https://numpy.org/doc/stable/reference/generated/numpy.argmin.html)), we can then display an image where the value of each pixel corresponds to the end member with the smallest angle for each pixel - in other words, the best match.

In [ ]:
fig, ax = plt.subplots(1, 1)

# this will tell us which end member has the smallest spectral angle for each pixel
best_match = np.argmin(all_angles, axis=2)

# this will set the nodata pixels to be masked so that we don't see them in the plot
best_match = np.ma.masked_array(best_match, mask=ds['reflectance'].loc[8] == 0)

ax.imshow(best_match)

Here, we can see that the Lake has a fairly uniform value of 0, indicating that we've done a good job classifying the water. We can also see that in the northern part of the image, we have a number of areas where we've successfully picked out the stands of ponderosa pines (value of 1). While most of the image has been classified as sandstone, we know from the [geologic maps](http://data.azgs.az.gov/geologic-map-of-arizona/) of the area that this isn't completely correct - we should see areas of basalts and other types of bedrock.

To get a better classification of the image, we would want to be sure to pick out additional end members, and possibly even include multiple samples - remember that small differences in grain size or chemical composition can have a large impact on the spectral signature. Remember that you can add additional end members from the [USGS Spectral Library](https://www.usgs.gov/labs/spectroscopy-lab/usgs-spectral-library) by downloading the data files and using `data/combine_spectral_library.py` to add the new end members to `data/spectral_library.csv`.

While these results aren't bad for an initial attempt, the image is still relatively noisy. To help reduce the impact of the noise in the data, we might also consider using the minimum noise fraction or a similar data reduction technique.

### 4.3 Exporting the results

As a last step, we'll export this classified image to a raster file using `rasterio`:

In [ ]:
import rasterio as rio

with rio.open('EO1H0380352003173110PF_RGB.tif', 'r') as src:
    profile = src.profile
    profile.update({'dtype': np.uint8, 'count': 1, 'nodata': 255})
    with rio.open('sam_classification.tif', 'w', **profile) as dst:
        dst.write(best_match.astype(np.uint8), 1)

You should be able to load the classified raster in a GIS software such as ArcGIS Pro or QGIS, and calculate the area and percent coverage of each end member in the image. Alternatively, you could also do this by editing the notebook. :)

That will wrap us up for this week's practical - as mentioned, there are a number of additional algorithms available from `spectral` - for a full list, check out the full [API documentation](https://www.spectralpython.net/class_func_ref.html).